<a href="https://colab.research.google.com/github/oriolgarrobe/text-mining/blob/master/TM_L5_Erik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

We start by loading spaCy:

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [ ]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(' '))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [ ]:
import bz2
import csv
import pandas as pd

with bz2.open('ner-train.tsv.bz2', 'rt') as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

with bz2.open('ner-dev.tsv.bz2', 'rt') as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [ ]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [ ]:
def evaluation_report(gold, pred):
    """Print precision, recall, and F1 score.
    
    Args:
        gold: The set with the gold-standard values.
        pred: The set with the predicted values.
    
    Returns:
        Nothing, but prints the precision, recall, and F1 values computed
        based on the specified sets.
    """
    # TODO: Replace the next line with your own code
    
    tp = gold.intersection(pred) # true positives
    
    precision = len(tp)/len(pred)
    
    recall = len(tp)/len(gold)
    
    f1 = 2*precision*recall/(precision+recall)
    
    print(f'Precision: {round(precision*100)}%\n')
    print(f'Recall: {round(recall*100)}%\n')
    print(f'F1-score: {round(f1*100)}%')

To test your code, you can run the following cell:

In [ ]:
evaluation_report(set(range(3)), set(range(5)))

Precision: 60%

Recall: 100%

F1-score: 75%


This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [ ]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    # TODO: Replace the next line with your own code
    
    for row in df.itertuples():
        yield (row[1],row[3],row[4])

To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [ ]:
spans_dev_gold = set(gold_spans(df_dev))
spans_dev_gold_l = list(gold_spans(df_dev))

print(len(spans_dev_gold))
print(spans_dev_gold_l[0])
print(spans_dev_gold_l[5916])

5917
('0946-000', 2, 3)
('1161-010', 1, 3)


Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [ ]:
# TODO: Write code here to run and evaluate the spaCy NER on the development data
def predict(df):
    for row in df.itertuples():
        sentence = nlp(row[2])
        for ent in sentence.ents:
            start = sentence.text.split(' ').index(ent.text.split(' ')[0])
            yield (row[1], start, start+len(ent.text.split(' ')))

In [ ]:
preds_dev_gold = set(predict(df_dev)) #predictions of dev data

evaluation_report(spans_dev_gold, preds_dev_gold)

Precision: 52%

Recall: 67%

F1-score: 59%


## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Here is a function that prints the false positives as well as the false negatives spans for a data frame, given a reference set of gold-standard spans and a candidate set of predicted spans.

In [ ]:
from collections import defaultdict

def error_report(df, spans_gold, spans_pred):
    false_pos = defaultdict(list)
    for s, b, e in spans_pred - spans_gold:
        false_pos[s].append((b, e))
    false_neg = defaultdict(list)
    for s, b, e in spans_gold - spans_pred:
        false_neg[s].append((b, e))
    for row in df.drop_duplicates('sentence_id').itertuples():
        if row.sentence_id in false_pos or row.sentence_id in false_neg:
            print('Sentence:', row.sentence)
            for b, e in false_pos[row.sentence_id]:
                print('  FP:', ' '.join(row.sentence.split()[b:e]))
            for b, e in false_neg[row.sentence_id]:
                print('  FN:', ' '.join(row.sentence.split()[b:e]))

Use this function to inspect and analyse the errors that the automated prediction makes. Can you see any patterns? Base your analysis on the first 500 rows of the training data. Summarize your observations in a short text.

In [ ]:
# TODO: Write code here to do your analysis
df_base = df_train.head(500)
spans_gold = set(gold_spans(df_base))
spans_pred = set(predict(df_base))

In [ ]:
error_report(df_base, spans_gold, spans_pred)

Sentence: The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep .
  FP: Thursday
  FP: The European Commission
  FN: European Commission
Sentence: Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer .
  FP: Wednesday
  FP: the European Union 's
  FN: European Union
Sentence: He said further scientific study was required and if it was found that action was needed it should be taken by the European Union .
  FP: the European Union
  FN: European Union
Sentence: He said a proposal last month by EU Farm Commissioner Franz Fischler to ban sheep brains , spleens and spinal cords from the human and animal food chains was a highly specific and precautionary move to protect human health .
  FP: last month
  

*TODO: Write a short text that summarises the errors that you observed*

**Answer:**
* Predicted 'The European Comission' instead of 'European Comission'. Some predictions contain less words than the gold-label, however it is difficult to descifer which is the logic behind the prediction. 
* Predicted dates. The gold-label set does not contain dates, therefore they will be removed from the prediction. 
* Predicted numeric values. The gold-label set does not contain numeric values, they will also be removed.

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to post-process the output produced by spaCy. You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [ ]:
# TODO: Write code here to improve the span prediction from Problem 2
def predict_improved(df):
    wrong_label = ['DATE', 'CARDINAL', 'QUANTITY', 'PERCENT', 'MONEY', 'TIME', 'NAME', 'ORDINAL', 'PRODUCT']
    for row in df.itertuples():
        sentence = nlp(row[2])
        for ent in sentence.ents:
            if ent.label_ not in wrong_label:
                start = sentence.text.split(' ').index(ent.text.split(' ')[0])
                yield (row[1], start, start+len(ent.text.split(' ')))

In [ ]:
preds_dev_gold = set(predict_improved(df_dev)) #predictions of dev data

evaluation_report(spans_dev_gold, preds_dev_gold)

Precision: 84%

Recall: 67%

F1-score: 74%


In [ ]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the *predicted* start and end positions for each token span, rather than the gold positions. As the `label` of each span, you can use the special value `--NME--`.

In [ ]:
# TODO: Write code here to store the predicted spans in a new data frame
df_pred = pd.DataFrame(preds_dev_gold)
df_pred['label'] = '--NME--'
df_pred = df_pred.rename(columns={0: "sentence-id", 1: "beg", 2: "end"})
df_pred.head()

,sentence-id,beg,end,label
0,1103-003,5,7,--NME--
1,1051-011,25,27,--NME--
2,1046-006,19,21,--NME--
3,1000-015,41,42,--NME--
4,1017-000,4,6,--NME--


## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [ ]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        yield (row[1],row[3],row[4],row[5])

In [ ]:
spans_dev_gold = set(gold_mentions(df_dev))
spans_dev_gold_l = list(gold_mentions(df_dev))

print(len(spans_dev_gold))
print(spans_dev_gold_l[0])
print(spans_dev_gold_l[5916])

5917
('0946-000', 2, 3, 'Leicestershire_County_Cricket_Club')
('1161-010', 1, 3, '--NME--')


A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

<div class="alert alert-warning">
    Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.
</div>

In [ ]:
# TODO: Write code here to implement the baseline
def predict_linked(df):
    wrong_label = ['DATE', 'CARDINAL', 'QUANTITY', 'PERCENT', 'MONEY', 'TIME', 'NAME', 'ORDINAL', 'PRODUCT']
    for row in df.itertuples():
        sentence = nlp(row[2])
        for ent in sentence.ents:
            if ent.label_ not in wrong_label:
                start = sentence.text.split(' ').index(ent.text.split(' ')[0])
                label = '_'.join(ent.text.split(' '))
                yield (row[1], start, start+len(ent.text.split(' ')), label)

In [ ]:
preds_dev_gold = set(predict_linked(df_dev)) #predictions of dev data

evaluation_report(spans_dev_gold, preds_dev_gold)

Precision: 32%

Recall: 25%

F1-score: 28%


In [ ]:
df_pred = pd.DataFrame(preds_dev_gold)
df_pred = df_pred.rename(columns={0: "sentence-id", 1: "beg", 2: "end", 3:'label'})
df_pred.head()

,sentence-id,beg,end,label
0,1045-003,12,14,Security_Council
1,0947-002,7,8,English
2,1139-003,2,3,Wang
3,0960-007,17,18,3-6
4,1065-012,0,1,Clydebank


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on) as mentions of the entities (pages) that they link to. This allows us to harvest long lists of mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [ ]:
with bz2.open('kb.tsv.bz2', 'rt') as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [ ]:
df_kb.loc[df_kb.mention == 'Sweden']

,mention,entity,prob
17436,Sweden,Sweden,0.985768
17437,Sweden,Sweden_national_football_team,0.014173
17438,Sweden,Sweden_men's_national_ice_hockey_team,0.000059


As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing come first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [ ]:
# TODO: Write code here to implement the "most probable entity" method.

def el_method(df):    
    wrong_label = ['DATE', 'CARDINAL', 'QUANTITY', 'PERCENT', 'MONEY', 'TIME', 'NAME', 'ORDINAL', 'PRODUCT']
    for row in df.itertuples():
      sentence = nlp(row[2])
      for ent in sentence.ents:
        if ent.label_ not in wrong_label:
          start = sentence.text.split(' ').index(ent.text.split(' ')[0])
          df_ent = df_kb.entity[df_kb.mention == ent.text]
          mention = '--NME--'
          if df_ent.shape[0] > 0:
            mention = df_ent.iloc[0]
          yield (row[1], start, start+len(ent.text.split(' ')), mention)

In [ ]:
el_pred = set(el_method(df_dev))
evaluation_report(spans_dev_gold, el_pred)

Precision: 65%

Recall: 51%

F1-score: 57%


In [ ]:
df_pred_el = pd.DataFrame(el_pred)
df_pred_el = df_pred_el.rename(columns={0: "sentence-id", 1: "beg", 2: "end", 3:'label'})
df_pred_el.head()

,sentence-id,beg,end,label
0,1003-007,12,14,Dési_Bouterse
1,0994-010,0,1,Alexander_Lukashenko
2,1098-002,6,7,World_Boxing_Organization
3,0966-157,4,5,Cuba
4,1128-007,0,1,--NME--


## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier that is trained to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

with bz2.open('contexts.tsv.bz2') as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

In [ ]:
df_contexts.head()

,mention,entity,context
0,1970,UEFA_Champions_League,Cup twice the first in @ and the second in 1983
1,1970,FIFA_World_Cup,America 1975 and during the @ and 1978 World C...
2,1990 World Cup,1990_FIFA_World_Cup,Manolo represented Spain at the @
3,1990 World Cup,1990_FIFA_World_Cup,Hašek represented Czechoslovakia at the @ and ...
4,1990 World Cup,1990_FIFA_World_Cup,renovations in 1989 for the @ The present capa...


Note that, in each context, the position of the mention is indicated by the `@` symbol.

From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [ ]:
df_contexts.context[df_contexts.mention == 'Lincoln']

41465    Nebraska Concealed Handgun Permit In @ municip...
41466    Lazlo restaurants are located in @ and Omaha C...
41467    California Washington Overland Park Kansas @ N...
41468    City Missouri Omaha Nebraska and @ Nebraska It...
41469    by Sandhills Publishing Company in @ Nebraska USA
                               ...                        
41609                                      @ Leyton Orient
41610                    English division three Swansea @ 
41611    league membership narrowly edging out @ on goa...
41612                                          @ Cambridge
41613                                                   @ 
Name: context, Length: 149, dtype: object

In [ ]:
df_kb.loc[df_kb.mention == 'Sweden']

,mention,entity,prob
17436,Sweden,Sweden,0.985768
17437,Sweden,Sweden_national_football_team,0.014173
17438,Sweden,Sweden_men's_national_ice_hockey_team,0.000059


In [ ]:
df_contexts.entity[df_contexts.mention == 'Sweden'].unique()

array(['Sweden', 'Sweden_national_football_team',
       "Sweden_men's_national_ice_hockey_team"], dtype=object)

Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [ ]:
def predict_ent(df):
  wrong_label = ['DATE', 'CARDINAL', 'QUANTITY', 'PERCENT', 'MONEY', 'TIME', 'NAME', 'ORDINAL', 'PRODUCT']
  for row in df.itertuples():
    sentence = nlp(row[2])
    for ent in sentence.ents:
      if ent.label_ not in wrong_label:
        start = sentence.text.split(' ').index(ent.text.split(' ')[0])
        entity = ent.text
        yield (row[1], start, start+len(ent.text.split(' ')), entity)

df_pred_ent = pd.DataFrame(set(predict_ent(df_dev)))
df_pred_ent = df_pred_ent.rename(columns={0: "sentence_id", 1: "beg", 2: "end", 3:'label'})
df_pred_ent.head()


,sentence_id,beg,end,label
0,0947-002,7,8,English
1,1139-003,2,3,Wang
2,0960-007,17,18,3-6
3,0966-050,1,3,Isaac Viciosa
4,0972-025,0,5,R. Kaluwitharana b S. Waugh


In [ ]:
df_pred_ent[df_pred_ent.sentence_id == "1045-002"].head()

,sentence_id,beg,end,label
1726,1045-002,23,25,Middle East
1884,1045-002,13,14,Palestinian
1920,1045-002,8,9,Israel
4651,1045-002,0,2,Security Council


In [ ]:
df_dev[df_dev.sentence_id == "1045-002"].head()

,sentence_id,sentence,beg,end,label
2698,1045-002,Security Council members expressed concern on ...,0,2,United_Nations_Security_Council
2699,1045-002,Security Council members expressed concern on ...,8,9,Israel
2700,1045-002,Security Council members expressed concern on ...,13,14,State_of_Palestine
2701,1045-002,Security Council members expressed concern on ...,23,25,Middle_East


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

NB_dict = {}

for mention in df_pred_ent.label.unique():
  #mention_id = '_'.join(mention.split(' '))
  NB_dict[mention] = {}
  if df_contexts.context[df_contexts.mention==mention].shape[0] == 0:
    for s_id in df_pred_ent.sentence_id[df_pred_ent.label ==mention].iteritems():
      NB_dict[mention][s_id[1]] = '--NME--'

  if df_contexts.context[df_contexts.mention==mention].shape[0] == 1:
    for s_id in df_pred_ent.sentence_id[df_pred_ent.label ==mention].iteritems():
      NB_dict[mention][s_id[1]] = df_contexts.entity[df_contexts.mention==mention][0]
  
  if df_contexts.context[df_contexts.mention==mention].shape[0] > 1:
    X = df_contexts.context[df_contexts.mention == mention]
    y = df_contexts.entity[df_contexts.mention == mention]
    #class_prior=df_kb.loc[df_kb.mention == mention].sort_values('entity').prob
    pipe = Pipeline(steps=[('preprocessor', CountVectorizer()), ('model', MultinomialNB())])
    pipe.fit(X,y)

    for row in df_pred_ent.loc[df_pred_ent.label==mention].itertuples():
      NB_dict[mention][row[1]] = pipe.predict(df_dev.sentence[df_dev.sentence_id == row[1]])[0]


In [ ]:
df_pred_ent.loc[df_pred_ent.sentence_id=="1016-012"]
#df_pred_ent.itertuples()[0][0]

,sentence_id,beg,end,label
1073,1016-012,9,10,Israeli
2288,1016-012,13,14,Netanyahu
3871,1016-012,0,1,Levy


In [ ]:
df_pred_csd.loc[df_pred_csd.sentence_id=="1016-012"]

,sentence_id,beg,end,label
1,1016-012,13,14,--NME--
4113,1016-012,9,10,Israel
4629,1016-012,0,1,--NME--


In [ ]:
df_contexts.loc[df_contexts.mention == 'Levy']

,mention,entity,context


In [ ]:
def pred_csd(df):
  for row in df.itertuples():
    entity = NB_dict[row[4]][row[1]]
    yield (row[1], row[2], row[3], entity)

csd_pred = set(pred_csd(df_pred_ent))
df_pred_csd = pd.DataFrame(csd_pred)
df_pred_csd = df_pred_csd.rename(columns={0: "sentence_id", 1: "beg", 2: "end", 3:'label'})
df_pred_csd.head(50)

,sentence_id,beg,end,label
0,0966-121,1,3,--NME--
1,1016-012,13,14,--NME--
2,1034-000,0,1,Dutch_people
3,1017-006,5,6,--NME--
4,1155-013,1,2,Nazi_Germany
5,0991-010,9,10,--NME--
6,1128-007,0,1,--NME--
7,0984-008,15,16,--NME--
8,0980-003,2,3,Chicago
9,1089-010,17,18,Sweden


In [ ]:
evaluation_report(spans_dev_gold, csd_pred)

Precision: 31%

Recall: 25%

F1-score: 27%


You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1.

**This was the last lab in the Text Mining course. Congratulations!**

<div class="alert alert-info">
    Please read the section ‘General information’ on the ‘Labs’ page of the course website before submitting this notebook!
</div>